In [2]:
import pandas as pd 
import numpy as np
import re
import codecs
import nltk
from tqdm import tqdm
from nltk.corpus import stopwords 
nltk.download('stopwords')


from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, Embedding, LSTM, Lambda
import keras.backend as K
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import keras


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/saraltayal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


In [3]:
TRAIN_CSV = './train.csv'
glove = './glove.6B.100d'
# MODEL_SAVING_DIR = './'

In [4]:
train_df = pd.read_csv(TRAIN_CSV)
train_df.dropna()
np.shape(train_df)
train_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [5]:
def load_glove_model(glove_file):

    f = codecs.open(glove_file + ".txt", 'r', encoding='utf-8')
    model = {}
    for line in f:
        split_line = line.split()
        word = split_line[0]
        embedding = np.array([float(val) for val in split_line[1:]])
        model[word] = embedding
    return model
def get_w2v(sentence, model):
    return np.array([model.get(val, np.zeros(100)) for val in sentence.split()], dtype=np.float64)

#build the model-> a bit slow ~30seconds 
model = load_glove_model(glove)


In [6]:
def preprocess(input, maxLen = 150):
    if type(input) != type("string"):
        print(input)
        return "skip" 
    input = input.lower()
    input = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", input)
    input = re.sub(r"what's", "what is ", input)
    input = re.sub(r"\'s", " ", input)
    input = re.sub(r"\'ve", " have ", input)
    input = re.sub(r"can't", "cannot ", input)
    input = re.sub(r"n't", " not ", input)
    input = re.sub(r"i'm", "i am ", input)
    input = re.sub(r"\'re", " are ", input)
    input = re.sub(r"\'d", " would ", input)
    input = re.sub(r"\'ll", " will ", input)
    input = re.sub(r",", " ", input)
    input = re.sub(r"\.", " ", input)
    input = re.sub(r"!", " ! ", input)
    input = re.sub(r"\/", " ", input)
    input = re.sub(r"\^", " ^ ", input)
    input = re.sub(r"\+", " + ", input)
    input = re.sub(r"\-", " - ", input)
    input = re.sub(r"\=", " = ", input)
    input = re.sub(r"'", " ", input)
    input = re.sub(r"(\d+)(k)", r"\g<1>000", input)
    input = re.sub(r":", " : ", input)
    input = re.sub(r" e g ", " eg ", input)
    input = re.sub(r" b g ", " bg ", input)
    input = re.sub(r" u s ", " american ", input)
    input = re.sub(r"\0s", "0", input)
    input = re.sub(r" 9 11 ", "911", input)
    input = re.sub(r"e - mail", "email", input)
    input = re.sub(r"j k", "jk", input)
    input = re.sub(r"\s{2,}", " ", input)
    return input

In [7]:
train_df = pd.read_csv(TRAIN_CSV)

indexesToRemove = []

lenTrain = np.shape(train_df['question1'])[0]

for col in ['question1', 'question2']:
    for row in tqdm(range(10000)): #lenTrain
        processedText = preprocess(train_df[col][row], maxLen = maxlen)
        if processedText == "skip":
            indexesToRemove.append(row)
        train_df.at[row,col] = get_w2v(processedText, model)

print(indexesToRemove)
print(np.shape(train_df['question1'])[0])
train_df = train_df.drop(indexesToRemove)
print(np.shape(train_df['question1'])[0])

100%|██████████| 10000/10000 [00:01<00:00, 6041.29it/s][]
404290
404290



In [74]:
def padZeros(input, maxLen):
    if np.shape(input)[0] == maxLen:
        return input
    lenToPad = maxLen - np.shape(input)[0]
    output = np.concatenate((input, np.zeros((lenToPad, 100))))
    return np.transpose(output)


In [75]:
left = train_df['question1'][0:10000].values
right = train_df['question2'][0:10000].values
Y = train_df['is_duplicate'][0:10000].values

maxlen = 35
indextracker = []

for index in range(len(right)):
    if len(left[index]) > maxlen or len(right[index]) > maxlen or len(left[index]) < 3 or len(right[index]) < 3:
        indextracker.append(index)
left = np.delete(left, indextracker)
right = np.delete(right, indextracker)
Y = np.delete(Y, indextracker)
assert len(left) == len(right) and len(left) == len(Y)
print(len(right))

#probably need to delete variables etc for memory lol


9875


In [76]:
left = [padZeros(elem, maxlen) for elem in left]
right = [padZeros(elem, maxlen) for elem in right]


In [82]:
idx = np.random.randint(len(right))
print(left[idx])
print(right[idx])
print(Y[idx])

[[-0.23769  -0.080693 -0.54264  ...  0.        0.        0.      ]
 [ 0.59392  -0.22378   0.41476  ...  0.        0.        0.      ]
 [ 0.58697  -0.64035   1.0322   ...  0.        0.        0.      ]
 ...
 [-0.58302  -0.55737  -1.2969   ...  0.        0.        0.      ]
 [ 0.20035   0.37175   0.76217  ...  0.        0.        0.      ]
 [ 0.50308   0.42807   0.46349  ...  0.        0.        0.      ]]
[[-0.23769  -0.26703  -0.43734  ...  0.        0.        0.      ]
 [ 0.59392   0.44911  -0.09405  ...  0.        0.        0.      ]
 [ 0.58697   0.55478   0.36665  ...  0.        0.        0.      ]
 ...
 [-0.58302  -0.87247   0.034775 ...  0.        0.        0.      ]
 [ 0.20035   0.83828   0.3189   ...  0.        0.        0.      ]
 [ 0.50308   0.465    -0.76246  ...  0.        0.        0.      ]]
1
